In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image, UnidentifiedImageError, ImageFile
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import math



# Part 2: Data Preprocessing and Augmentation

# Image Data Generators for training and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 80% training, 20% validation split
)

# Define paths and image settings
train_dir = '/Users/grose/Desktop/dog_cat_cleaned'
target_size = (224, 224)  # Image size for the CNN model
batch_size = 32

# Custom generator to handle corrupt images
def safe_flow_from_directory(datagen, directory, target_size, batch_size, class_mode, subset):
    generator = datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        subset=subset
    )
    
    while True:
        try:
            yield next(generator)  # Yield batch
        except (OSError, UnidentifiedImageError) as e:
            print(f"Skipping corrupt image: {e}")
            continue  # Skip and move on to the next batch

# Training and validation data generators
train_generator = safe_flow_from_directory(
    datagen, train_dir, target_size=target_size, batch_size=batch_size, class_mode='binary', subset='training'
)

validation_generator = safe_flow_from_directory(
    datagen, train_dir, target_size=target_size, batch_size=batch_size, class_mode='binary', subset='validation'
)

# Calculate steps per epoch
steps_per_epoch = math.ceil(22440 / batch_size)  # Update 22440 with your actual training set size
validation_steps = math.ceil(5610 / batch_size)  # Update 5610 with your actual validation set size

# Part 3: Building a Custom CNN Model

# Create the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),  # To prevent overfitting
    layers.Dense(1, activation='sigmoid')  # Binary classification (cats and dogs)
])

# Compile the CNN model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary to see the structure
model.summary()

# Part 4: Training the Custom CNN Model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the CNN model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,  # You can adjust this
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)

# Part 5: Evaluating the CNN Model

# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")

# Part 6: Plotting Training and Validation Accuracy and Loss

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 5))

    # Plot training and validation accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

# Plot the history
plot_history(history)

# Part 7: Confusion Matrix and Classification Report

# Get true labels and predictions
true_labels = []
predicted_labels = []

for i in range(validation_steps):
    images, labels = next(validation_generator)
    true_labels.extend(labels)
    predictions = model.predict(images)
    predicted_labels.extend(predictions > 0.5)  # Convert probabilities to binary predictions

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['cat', 'dog'], yticklabels=['cat', 'dog'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Classification report
report = classification_report(true_labels, predicted_labels, target_names=['cat', 'dog'])
print(report)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0